In [1]:
import warnings
warnings.filterwarnings('ignore')

# Day 16 - It starts with a tensor

## Tensors: Multidimensional arrays

### From Python lists to PyTorch tensors

In [2]:
a = [1.0, 2.0, 1.0]

In [3]:
a[0]

1.0

In [4]:
a[2] = 3.0
a

[1.0, 2.0, 3.0]

### Constructing our first tensors

In [5]:
import torch
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [6]:
a[1]

tensor(1.)

In [7]:
float(a[1])

1.0

In [8]:
a[2] = 2.0
a

tensor([1., 1., 2.])

### The essence of tensors

* Python lists and tuples of numbers are stored as individual objects in memory
* A PyTorch tensors (and NumPy nparrays) are views over (typically) contiguous memory blocks
* A 1D tensor of 1,000,000 32-bit floats will take up 4,000,000 contiguous bytes (+ metadata)

In [9]:
# A triangle, with x and y coordinates of each point stored alternatingly
# (4, 1), (5, 3), (2, 1)
points = torch.zeros(6)
points[0] = 4.0
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0

In [10]:
# Alternate expression
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

In [11]:
# First point
float(points[0]), float(points[1])

(4.0, 1.0)

* It would be much more practical to store these as actual pairs
* We can do so with a 2D tensor
* The first index refers to points

In [12]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [13]:
points.shape

torch.Size([3, 2])

* We can also pass a size as arguments to the `torch.zeros` constructor

In [14]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

* The second index refers to individual coordinates

In [15]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [16]:
points[0, 1]

tensor(1.)

In [17]:
points[0]

tensor([4., 1.])

* The outputs that we are seeing here are just different views over the same contiguous data
* They are separate tensors, but view the same underlying memory

## Indexing tensors

* Just like Python lists can be indexed with ranges, so can PyTorch tensors

In [18]:
some_list = list(range(6))
print(some_list[:])
print(some_list[1:4])
print(some_list[1:])
print(some_list[:4])
print(some_list[:-1])
print(some_list[1:4:2])

[0, 1, 2, 3, 4, 5]
[1, 2, 3]
[1, 2, 3, 4, 5]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[1, 3]


* PyTorch tensors have the advantage that they can be indexed by multiple, comma-separated ranges

In [19]:
print(points[1:])
print(points[1:, :])
print(points[1:, 0])
print(points[None])

tensor([[5., 3.],
        [2., 1.]])
tensor([[5., 3.],
        [2., 1.]])
tensor([5., 2.])
tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])


* PyTorch also allows for $advanced\ indexing$, which we will have a look at later

## Broadcasting

* PyTorch support's NumPy's broadcasting semantics
* Dimensions are compared right-to-left
* Two dimensions are compatible if at least one of these is true:
    * The dimensions are equal
    * One of the dimensions is 1

In [20]:
# Tensor X:                 Scalar Y:
torch.tensor([[ 1, 2, 3 ]]) + 10

tensor([[11, 12, 13]])

* A 1x3 tensor can be broadcast with a 3x1 tensor, as for each dimension, one of them is 1
* The result is a 3x3 tensor

|3|x|3|
|-|-|-|
|3|x|1|
|1|x|3|

In [21]:
torch.tensor([1, 2, 3]) * torch.tensor([[10],
                                        [20],
                                        [30]])

tensor([[10, 20, 30],
        [20, 40, 60],
        [30, 60, 90]])

* For varying dimensionality, the shapes are right-aligned and compared

|2|x|2|x|3|
|-|-|-|-|-|
|2|x|1|x|3|
| | |2|x|3|

In [22]:
A = torch.tensor([
    [1, 2, 3],
    [4, 5, 6],
])
B = torch.tensor([
    [[10, 20, 30]], # Note the extra brackets
    [[40, 50, 60]], # This tensor is of shape (2, 1, 3)
])

A + B

tensor([[[11, 22, 33],
         [14, 25, 36]],

        [[41, 52, 63],
         [44, 55, 66]]])

## Named tensors

* Often, the axes of a tensor have distinct meanints, like color channel, or row of pixels
* Having to remember these can be highly error-prone

In [23]:
# Example: image
img_t = torch.randn(3, 5, 5) # shape (channel, rows, columns)
weights = torch.tensor([0.2126, 0.7152, 0.0722]) # Typical color channel weights

* We often want to generalized, for example from 2D greyscale images to 3D RGB, or from a single image to a batch of images

In [24]:
batch_t = torch.randn(2, 3, 5, 5) # shape (batch, channels, rows, columns)

* Sometimes the RGB channels are in dimension 0, sometimes in dimension 1
* They are, however, always -3 from the end

In [25]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

* Things become hard to follow once the weights are added

In [26]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1)
img_weights = img_t * unsqueezed_weights
batch_weights = batch_t * unsqueezed_weights
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

* To help with this difficulty, which quickly leads to errors, $named\ tensors$ were introduced in PyTorch 1.3

In [27]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=["channels"])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

* With the `refine_names` method, names can be added, without changing existing names
* The `rename` method does change existing names, and allows dropping them (by using `None`)

In [28]:
# Leading dimensions ignored -> ...
img_named =  img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


* Names are now checked during operations between two tensors, in addition to dimensions
* To align dimensions, we can use `align_as`, since this is not done automatically yet

In [29]:
weights_named.shape, weights_named.names

(torch.Size([3]), ('channels',))

In [30]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names 

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

* The `'rows'` and `'columns'` dimensions are added in their respective positions

* Functions like `sum()`, which accept dimensions, also accept their names as arguments

In [31]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

* Trying to combine dimensions of different names is an error

In [32]:
try:
    gray_named = (img_named[..., :3] * weights_named).sum('channels')
except RuntimeError as e:
    print(e)

Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.


* To continue using tensors outside of functions that operate on named tensors, we can drop their names with `rename`
* As named tensors are still experimental (the warning is suppressed in this notebook, so check whether they still are), we will continue with unnamed tensors for now

In [33]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))